In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

!hdfs dfs -D dfs.replication=1 -cp -f data/*.csv hdfs://nn:9000/
!hdfs dfsadmin -fs hdfs://nn:9000/ -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 7914276941 (7.37 GB)
DFS Remaining: 7734190080 (7.20 GB)
DFS Used: 180086861 (171.74 MB)
DFS Used%: 2.28%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.19.0.2:9866 (project-5-lukee5-dn-1.project-5-lukee5_default)
Hostname: c7ec944d6b1f
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 180086861 (171.74 MB)
Non DFS Used: 17889998771 (16.66 GB)
DFS Remaining: 7734190080 (7.20 GB)
DFS Used%: 0.70%
DFS Remaining%: 29.

In [2]:
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/05 23:05:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
banks_df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/arid2017_to_lei_xref_csv.csv"))
banks_rdd = banks_df.rdd
rows = banks_rdd.take(3)
# consider using code to extract header name from rows
rows

[Row(respondent_name='First National Bank', arid_2017='110004', lei_2018='5493003EW6T31TGECO83', lei_2019='5493003EW6T31TGECO83', lei_2020='5493003EW6T31TGECO83'),
 Row(respondent_name='First Mid Bank & Trust, National Association', arid_2017='110045', lei_2018='549300XOTES5TCS8T794', lei_2019='549300XOTES5TCS8T794', lei_2020='549300XOTES5TCS8T794'),
 Row(respondent_name='First Hope Bank, A National Banking Association', arid_2017='110118', lei_2018='5493003XLOX5FDT9R120', lei_2019='5493003XLOX5FDT9R120', lei_2020='5493003XLOX5FDT9R120')]

In [4]:
#q1
the_national = lambda row: "The" in row["respondent_name"] and "National" in row["respondent_name"]
num_banks = banks_rdd.filter(the_national).count()
num_banks

106

In [5]:
#q2
num_banks2 = banks_df.filter("respondent_name LIKE '%The%' AND respondent_name LIKE '%National%'").count()
num_banks2

106

In [6]:
banks_df.write.saveAsTable("banks", mode="overwrite")

24/04/05 23:06:14 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/05 23:06:14 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/05 23:06:18 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/05 23:06:18 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.28.0.5
24/04/05 23:06:20 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/04/05 23:06:24 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/04/05 23:06:24 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/05 23:06:24 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/05 23:06:24 W

In [7]:
#q3
spark.sql("SELECT COUNT(*) AS num_banks FROM banks WHERE respondent_name LIKE '%The%' AND respondent_name LIKE '%National%'").collect()[0]['num_banks']

106

In [8]:
loans_df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/hdma-wi-2021.csv"))
loans_df.write.bucketBy(8, "county_code").saveAsTable("loans", mode="overwrite")
#!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse/loans

24/04/05 23:06:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
views_to_create = [
    "ethnicity", "race", "sex", "states", "counties", "tracts", "action_taken",
    "denial_reason", "loan_type", "loan_purpose", "preapproval", "property_type"]

for view_name in views_to_create:
    view_df = spark.read.csv(f"hdfs://nn:9000/{view_name}.csv", header=True, inferSchema=True)
    view_df.createOrReplaceTempView(view_name)

In [10]:
tables_df = spark.sql("SHOW TABLES")
tables_df.show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|        banks|      false|
|  default|        loans|      false|
|         | action_taken|       true|
|         |     counties|       true|
|         |denial_reason|       true|
|         |    ethnicity|       true|
|         | loan_purpose|       true|
|         |    loan_type|       true|
|         |  preapproval|       true|
|         |property_type|       true|
|         |         race|       true|
|         |          sex|       true|
|         |       states|       true|
|         |       tracts|       true|
+---------+-------------+-----------+



In [11]:
#q4
dict = {}
tables_pd = tables_df.toPandas()
for index, row in tables_pd.iterrows():
    dict[row.iloc[1]] = row.iloc[2]
dict

{'banks': False,
 'loans': False,
 'action_taken': True,
 'counties': True,
 'denial_reason': True,
 'ethnicity': True,
 'loan_purpose': True,
 'loan_type': True,
 'preapproval': True,
 'property_type': True,
 'race': True,
 'sex': True,
 'states': True,
 'tracts': True}

In [12]:
spark.sql("SELECT * FROM banks").show()

+--------------------+---------+--------------------+--------------------+--------------------+
|     respondent_name|arid_2017|            lei_2018|            lei_2019|            lei_2020|
+--------------------+---------+--------------------+--------------------+--------------------+
| First National Bank|   110004|5493003EW6T31TGECO83|5493003EW6T31TGECO83|5493003EW6T31TGECO83|
|First Mid Bank & ...|   110045|549300XOTES5TCS8T794|549300XOTES5TCS8T794|549300XOTES5TCS8T794|
|First Hope Bank, ...|   110118|5493003XLOX5FDT9R120|5493003XLOX5FDT9R120|5493003XLOX5FDT9R120|
|First National Ba...|   110180|5493002GOGIUXB84JB89|5493002GOGIUXB84JB89|5493002GOGIUXB84JB89|
|Cumberland Valley...|   110254|549300OULGV8ZW0JV320|549300OULGV8ZW0JV320|549300OULGV8ZW0JV320|
|The Lyons Nationa...|    11027|254900Q6IHGQO7INVC79|254900Q6IHGQO7INVC79|254900Q6IHGQO7INVC79|
|Midstates Bank, N...|   110354|2549002IXACRD1HE6A92|2549002IXACRD1HE6A92|2549002IXACRD1HE6A92|
|First National Ba...|   110406|5493000L

In [13]:
#q5
banks_loans_df = spark.sql("""
    SELECT loans.*, banks.respondent_name
    FROM loans
    INNER JOIN banks ON loans.lei = banks.lei_2020
""")
num_loan_applications = banks_loans_df.filter("respondent_name = 'First National Bank'").count()
num_loan_applications

6

In [14]:
banks_loans_df.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (8)
+- Project (7)
   +- BroadcastHashJoin Inner BuildRight (6)
      :- Filter (2)
      :  +- Scan parquet spark_catalog.default.loans (1)
      +- BroadcastExchange (5)
         +- Filter (4)
            +- Scan parquet spark_catalog.default.banks (3)


(1) Scan parquet spark_catalog.default.loans
Output [99]: [activity_year#1025, lei#1026, derived_msa-md#1027, state_code#1028, county_code#1029, census_tract#1030, conforming_loan_limit#1031, derived_loan_product_type#1032, derived_dwelling_category#1033, derived_ethnicity#1034, derived_race#1035, derived_sex#1036, action_taken#1037, purchaser_type#1038, preapproval#1039, loan_type#1040, loan_purpose#1041, lien_status#1042, reverse_mortgage#1043, open-end_line_of_credit#1044, business_or_commercial_purpose#1045, loan_amount#1046, loan_to_value_ratio#1047, interest_rate#1048, rate_spread#1049, hoepa_status#1050, total_loan_costs#1051, total_points_and_fees#1052, origination_charges#1053, discount_

In [15]:
#q6
# 1. Banks. In the output above, BroadcastExchange, which is a part of BroadcastHashJoin, sends the banks table to every executor
# 2. No, the plan does not involve HashAggregates. I know this because from the output above, we see that the query doesn't use any aggregate functions, such as COUNT, SUM, or AVG

In [16]:
spark.sql("SELECT * FROM counties WHERE 1=0").show()
spark.sql("SELECT * FROM loans WHERE 1=0").show()

+----+---------+---------+----------+-----+------+----+----+----------+
|AREA|PERIMETER|CO99_D00_|CO99_D00_I|STATE|COUNTY|NAME|LSAD|LSAD_TRANS|
+----+---------+---------+----------+-----+------+----+----+----------+
+----+---------+---------+----------+-----+------+----+----+----------+

+-------------+---+--------------+----------+-----------+------------+---------------------+-------------------------+-------------------------+-----------------+------------+-----------+------------+--------------+-----------+---------+------------+-----------+----------------+-----------------------+------------------------------+-----------+-------------------+-------------+-----------+------------+----------------+---------------------+-------------------+---------------+--------------+---------+-----------------------+-----------------+---------------------+---------------------+---------------+----------------------------+--------------+-------------------+--------------+-------------------------

In [17]:
#q7
wells_fargo = spark.sql("""
    SELECT loans.*, banks.respondent_name
    FROM loans
    INNER JOIN banks ON loans.lei = banks.lei_2020
    WHERE banks.respondent_name LIKE '%Wells Fargo%'
""")

avg = wells_fargo.groupBy("county_code").avg("loan_amount")

top_ten = avg.join(
    spark.table("counties"),
    avg.county_code == (col("counties.STATE") * 1000 + col("counties.NAME"))
).orderBy(col("avg(loan_amount)").desc()).limit(10)

apps = wells_fargo.groupBy("county_code").count()

# i'm really close on this one but I couldn't figure out how to join apps and top_ten on name without receiving an exception
counties = apps.join(
    top_ten,
    apps.county_code == top_ten.county_code
).select("counties.NAME", "count")

wells_fargo_dict = {}
wells_fargo_pd = apps.toPandas()
for index, row in wells_fargo_pd.iterrows():
    wells_fargo_dict[row.iloc[0]] = row.iloc[1]
wells_fargo_dict

{'55127': 282,
 '55005': 58,
 '55015': 125,
 '55089': 389,
 '55013': 62,
 '55007': 33,
 '55099': 13,
 '55065': 12,
 '55023': 8,
 '55109': 289,
 '55061': 22,
 '55131': 748,
 '55117': 307,
 '55033': 43,
 '55049': 26,
 '55047': 30,
 '55097': 44,
 '55081': 35,
 '55111': 108,
 '55021': 79,
 '55063': 129,
 '55087': 513,
 '55069': 35,
 '55045': 23,
 '55001': 46,
 '55115': 81,
 '55003': 17,
 '55078': 7,
 '55077': 29,
 '55059': 271,
 '55139': 402,
 '55067': 26,
 '55075': 49,
 '55027': 129,
 '55031': 78,
 '55105': 181,
 '55085': 70,
 '55017': 122,
 '55011': 33,
 '55057': 27,
 '55073': 153,
 '55123': 16,
 '55119': 17,
 '55137': 43,
 '55129': 39,
 '55121': 19,
 '55101': 537,
 '55083': 99,
 '55141': 44,
 '55019': 20,
 '55051': 23,
 '55079': 2164,
 '55025': 729,
 '55071': 116,
 '55095': 95,
 '55093': 80,
 '55039': 163,
 '55055': 131,
 '55035': 160,
 '55107': 33,
 '55029': 87,
 '55043': 19,
 '55113': 38,
 '55037': 8,
 '55091': 10,
 '55053': 10,
 '55009': 663,
 '55135': 72,
 '55133': 1832,
 '55125': 6

In [18]:
group_by_county_code = spark.sql("""
    SELECT county_code, AVG(loan_amount) AS avg_loan_amount
    FROM loans
    GROUP BY county_code
""")
group_by_county_code.explain("formatted")

# Grouping by lei column
group_by_lei = spark.sql("""
    SELECT lei, AVG(loan_amount) AS avg_loan_amount
    FROM loans
    GROUP BY lei
""")
group_by_lei.explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (4)
+- HashAggregate (3)
   +- HashAggregate (2)
      +- Scan parquet spark_catalog.default.loans (1)


(1) Scan parquet spark_catalog.default.loans
Output [2]: [county_code#1029, loan_amount#1046]
Batched: true
Bucketed: true
Location: InMemoryFileIndex [hdfs://nn:9000/user/hive/warehouse/loans]
ReadSchema: struct<county_code:string,loan_amount:double>
SelectedBucketsCount: 8 out of 8

(2) HashAggregate
Input [2]: [county_code#1029, loan_amount#1046]
Keys [1]: [county_code#1029]
Functions [1]: [partial_avg(loan_amount#1046)]
Aggregate Attributes [2]: [sum#2351, count#2352L]
Results [3]: [county_code#1029, sum#2353, count#2354L]

(3) HashAggregate
Input [3]: [county_code#1029, sum#2353, count#2354L]
Keys [1]: [county_code#1029]
Functions [1]: [avg(loan_amount#1046)]
Aggregate Attributes [1]: [avg(loan_amount#1046)#2348]
Results [2]: [county_code#1029, avg(loan_amount#1046)#2348 AS avg_loan_amount#2347]

(4) AdaptiveSparkPlan
Output [2]: [county_co

In [19]:
#q8
# Network I/O is needed to exchange data between nodes.
# As seen in the explanation of these queries, grouping by county_code does not require an Exchange step, but grouping by lei does. 
# Therefore, network I/O is necessary for grouping by lei because the associated query uses an Exchange step.
# Network I/O is not necessary for grouping by county_code because the associated query does not use an Exchange step

In [20]:
#q9
df = spark.sql("""
    SELECT CAST(loan_amount AS DOUBLE) AS loan_amount,
           CAST(income AS DOUBLE) AS income,
           CAST(interest_rate AS DOUBLE) AS interest_rate,
           CAST((action_taken = 'Loan originated') AS DOUBLE) AS approval
    FROM loans
""")

filled_df = df.fillna(0.0, subset=['loan_amount', 'income', 'interest_rate', 'approval'])
train, test = filled_df.randomSplit([0.8, 0.2], seed=41)

train.write.format("parquet").mode("ignore").save("hdfs://nn:9000/train.parquet")
test.write.format("parquet").mode("ignore").save("hdfs://nn:9000/test.parquet")

train = spark.read.format("parquet").load("hdfs://nn:9000/train.parquet")
test = spark.read.format("parquet").load("hdfs://nn:9000/test.parquet")

train.cache()

num_approved_loans_train = train.filter(col("approval") == 1).count()

# not sure why I'm only getting 0; my code looks right as far as I can tell
num_approved_loans_train

0

In [21]:
#q10
# I ran into a lot of problems on this one because I was having difficulty getting the correct answer for the previous question